## Google Dance

In [1]:
#Importing all the libraries required for scraping

from splinter import Browser
from bs4 import BeautifulSoup 
import requests
import time
import pandas as pd
import pprint
import webbrowser
import urllib
import re
import pymongo
from geopy.geocoders import Nominatim  

In [9]:
# using the crome driver for scraping

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
#browser = Browser()

In [3]:
# Url for main page
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=Y5HcXZW5CdG05gKT7KzIAw&q=kids+dance+classes+new+jersey&oq=kids+dance+classes+new+jersey&gs_l=psy-ab.3...140911.143275.0.143677.6.6.0.0.0.0.271.904.0j4j1.5.0....0...1c.1.64.psy-ab..1.0.0....0.2IUp-hMGy8U#rlfi=hd:;si:;mv:[[41.0322952,-73.93508229999999],[40.534548400000006,-74.637359]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!2m1!1e2!2m1!1e3!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

# updating the page by clicking on search button
browser.find_by_name('btnG').first.click()

# parser
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

# storing all the data from the main class
Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
# search variables Name and reviews within main class using for loop
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'The Art of Dance'}, {'name': 'Arya Dance Academy'}, {'name': 'Wild Child Dance Studio'}, {'name': 'Class Act Performing Arts Studio'}, {'name': 'Hudson Dance Studio Edgewater'}, {'name': 'Harmony Dance Center'}, {'name': 'Summit School of Dance'}, {'name': 'Kaye Lynn Dance Studio I'}, {'name': "Sharon's Studio of Dance & Music"}, {'name': 'Dance Palooza Studios'}, {'name': 'Village Dance Academy'}, {'name': 'Grooves Unlimited Dance Studio'}, {'name': 'New Jersey Ballet Company and School'}, {'name': 'Cherney Dance Studio'}, {'name': 'Dance Academy of North Jersey'}, {'name': 'American Dance Studios /Summer Camp/Edison/Piscataway/Metuchen/NewBrunswickNJ'}, {'name': 'Edison School of Music and Dance'}, {'name': 'Lydia Johnson Dance School'}, {'name': 'Starlight Dance Center'}, {'name': 'Across the Floor'}, {'name': 'The Kennedy Dancers'}, {'name': 'bboy bgirl lifestyle'}]


In [4]:

Data = []
# Using the geo locator for getting Lat and Long
geolocator = Nominatim(user_agent="my-application")

# getting the link for next pages 

pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

#updating each page link with specific nunmbers to go to next page

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

# parsing pages as the pages updated with above url

    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))

# search variables Name, url, adress, phone no, zipcode within main class using for loop

    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Dance Classes"
        
#if the url is empty handling the exception

        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            print(dataframe["url"])
        except:
            print('Cant Find the url')

# scraping for phone number

        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        
# if the main directory is empty, handling the exception
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
        
# getting the address
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])

# cleaning the address based on number of delimters

            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                #    print(orgAddress)
                
# Using geolocator to find the lat and long for each address

            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)

# creating a dataframe for Dance
df1=pd.DataFrame(top_result)



1
/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=0KfmXen8CfCg_Qbl5ay4Cg&start=20&sa=N&ved=0ahUKEwjppICxjJrmAhVwUN8KHeUyC6cQ8tMDCPoC
2
start=20
start=40
https://www.google.com/search?q=kids+dance+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=0KfmXen8CfCg_Qbl5ay4Cg&start=40&sa=N&ved=0ahUKEwjppICxjJrmAhVwUN8KHeUyC6cQ8tMDCPoC
20
http://www.dancetheatreofnewjersey.com/
Dance+Theatre+of+New+Jersey,+112+Bauer+Dr,+Oakland,+NJ+07436
['Dance Theatre of New Jersey', ' 112 Bauer Dr', ' Oakland', ' NJ 07436']
 112 Bauer Dr,  Oakland, NJ, 07436
112, Bauer Drive, Oakland, Bergen County, New Jersey, 07436, United States of America
(41.0061708789771, -74.240482416169)
http://www.newarkschoolofthearts.org/
Newark+School+of+the+Arts,+89+Lincoln+Park,+Newark,+NJ+07102
['Newark School of the Arts', ' 89 Lincoln Park', ' Newark', ' NJ 07102']
 89 Lincoln Park,  Newark, NJ, 07102
89, Lincoln Park, Teachers Village, Newark, Essex County, New Jersey, 07102, Unite

1966, Washington Valley Road, Bridgewater Township, Somerset County, New Jersey, 08836, United States of America
(40.6011205012849, -74.5612340964691)
http://www.beyonddancenj.com/
Beyond+Dance,+123+N+Union+Ave,+Cranford,+NJ+07106
['Beyond Dance', ' 123 N Union Ave', ' Cranford', ' NJ 07106']
 123 N Union Ave,  Cranford, NJ, 07106
North Union Avenue, Cranford, Union County, New Jersey, 07016, United States
(40.6555406, -74.3045367)
http://www.bergencountydanceclasses.com/
Broadway+Bound+Dance+Center+Inc.,+99+W+Madison+Ave,+Dumont,+NJ+07628
['Broadway Bound Dance Center Inc.', ' 99 W Madison Ave', ' Dumont', ' NJ 07628']
 99 W Madison Ave,  Dumont, NJ, 07628
99, West Madison Avenue, Dumont, Bergen County, New Jersey, 07628, United States
(40.94016, -73.9985329090909)
https://rogersdanceacademy.com/
Rogers+Dance+Academy,+97+River+Rd,+Flemington,+NJ+08822
['Rogers Dance Academy', ' 97 River Rd', ' Flemington', ' NJ 08822']
 97 River Rd,  Flemington, NJ, 08822
97, River Road, Raritan Towns

1929, Morris Avenue, Union, Union County, New Jersey, 07083, United States
(40.695712, -74.266432)
http://www.gotta-dance.com/
Gotta+Dance,+3322+US-22+Ste+207,+Bldg+2,+Branchburg,+NJ+08876
['Gotta Dance', ' 3322 US-22 Ste 207', ' Bldg 2', ' Branchburg', ' NJ 08876']
['', 'NJ', '08876']
 Bldg 2,  Branchburg, NJ, 08876
Cant Find the address
http://www.balletartsnj.com/
Ballet+Arts+NJ,+207+Kinderkamack+Rd,+Emerson,+NJ+07630
['Ballet Arts NJ', ' 207 Kinderkamack Rd', ' Emerson', ' NJ 07630']
 207 Kinderkamack Rd,  Emerson, NJ, 07630
Franco's Pizza, 207, Kinderkamack Road, Emerson, Bergen County, New Jersey, 07630, United States of America
(40.9763017, -74.0258807)
Cant Find the url
Dance+Xplosion,+601+Union+Blvd,+Totowa,+NJ+07512
['Dance Xplosion', ' 601 Union Blvd', ' Totowa', ' NJ 07512']
 601 Union Blvd,  Totowa, NJ, 07512
601, Union Boulevard, Totowa, Passaic County, New Jersey, 07512, United States
(40.893854, -74.225392)
http://www.aryaintl.com/
Arya+Dance+Academy,+1571+US-46,+Parsip

345, Morris Avenue, Elizabeth, Union County, New Jersey, 07208, United States
(40.669877, -74.218818)
http://www.tlglivingstonnj.com/
The+Little+Gym+of+Livingston,+4215+Town+Center+Way,+Livingston,+NJ+07039
['The Little Gym of Livingston', ' 4215 Town Center Way', ' Livingston', ' NJ 07039']
 4215 Town Center Way,  Livingston, NJ, 07039
Cant Find the address
http://www.kidville.com/hoboken
Kidville,+1202+Shipyard+Ln,+Hoboken,+NJ+07030
['Kidville', ' 1202 Shipyard Ln', ' Hoboken', ' NJ 07030']
 1202 Shipyard Ln,  Hoboken, NJ, 07030
Cant Find the address
http://warnockseeryirishdance.com/
Warnock+Seery+School+of+Irish+Dance,+231+Throckmorton+St+%231,+Freehold,+NJ+07728
['Warnock Seery School of Irish Dance', ' 231 Throckmorton St %231', ' Freehold', ' NJ 07728']
 231 Throckmorton St %231,  Freehold, NJ, 07728
Cant Find the address
http://www.gottadancenj.com/
Gotta+Dance,+285+Market+St,+Elmwood+Park,+NJ+07407
['Gotta Dance', ' 285 Market St', ' Elmwood Park', ' NJ 07407']
 285 Market St,

25, West Water Street, Toms River, Ocean County, New Jersey, 08753, United States
(39.9525654285714, -74.1998004489796)
http://www.kavischoolofdance.com/html/contact.htm
Kavi's+School+of+Dance,+125+Washington+Valley+Rd,+Warren,+NJ+07059
["Kavi's School of Dance", ' 125 Washington Valley Rd', ' Warren', ' NJ 07059']
 125 Washington Valley Rd,  Warren, NJ, 07059
Washington Valley Road, Warren Township, Somerset County, New Jersey, 07059, United States of America
(40.6007136, -74.5389071)
http://www.danceconnectionnj.com/
The+Dance+Connection,+1+Jill+Ct,+Hillsborough+Township,+NJ+08844
['The Dance Connection', ' 1 Jill Ct', ' Hillsborough Township', ' NJ 08844']
 1 Jill Ct,  Hillsborough Township, NJ, 08844
1, Jill Court, Hillsborough Township, Somerset County, New Jersey, 08844, United States
(40.492895, -74.641456)
https://jerseycityballroom.com/
Jersey+City+Ballroom,+189+Brunswick+St+3rd+FL,+Jersey+City,+NJ+07302
['Jersey City Ballroom', ' 189 Brunswick St 3rd FL', ' Jersey City', ' NJ

Cant Find the address
http://dansations.net/
Dansations+School+of+Dance,+1509+NJ-38,+Hainesport,+NJ+08036
['Dansations School of Dance', ' 1509 NJ-38', ' Hainesport', ' NJ 08036']
 1509 NJ-38,  Hainesport, NJ, 08036
1509, State Route 38, Crossroads Plaza, Hainesport Township, Burlington County, New Jersey, 08036, United States
(39.981854690155, -74.8114795159342)
https://www.misspatti.com/
Miss+Patti's+School+of+Dance,+85+Godwin+Ave+%23+32,+Midland+Park,+NJ+07432
["Miss Patti's School of Dance", ' 85 Godwin Ave %23 32', ' Midland Park', ' NJ 07432']
 85 Godwin Ave %23 32,  Midland Park, NJ, 07432
Cant Find the address
http://www.kidville.com/summit
Kidville,+30+Maple+St,+Summit,+NJ+07901
['Kidville', ' 30 Maple St', ' Summit', ' NJ 07901']
 30 Maple St,  Summit, NJ, 07901
30, Maple Street, Summit, Union County, New Jersey, 07901, United States
(40.7174857423286, -74.3587436861144)
http://alliedperformingarts.com/
Allied+Performing+Arts+Center,+6+Edgeboro+Rd,+East+Brunswick,+NJ+08816
['

555, Shrewsbury Avenue, Shrewsbury Township, Monmouth County, New Jersey, 07702, United States of America
(40.3333690925668, -74.0741926444282)
http://www.idakini.com/
Dakini+Movement+Studio,+51+Mt+Bethel+Rd,+Warren,+NJ+07059
['Dakini Movement Studio', ' 51 Mt Bethel Rd', ' Warren', ' NJ 07059']
 51 Mt Bethel Rd,  Warren, NJ, 07059
51, Mount Bethel Road, Warren Township, Somerset County, New Jersey, 07059, United States of America
(40.6204601694969, -74.4965213532099)
http://www.pranavamschoolofdance.com/
Pranavam+School+of+Dance,+204+Sayre+Dr,+Princeton,+NJ+08540
['Pranavam School of Dance', ' 204 Sayre Dr', ' Princeton', ' NJ 08540']
 204 Sayre Dr,  Princeton, NJ, 08540
Cant Find the address
https://www.fredastaire.com/manasquan
Fred+Astaire+Dance+Studios+-+Manasquan,+2410+NJ-35,+Manasquan,+NJ+08736
['Fred Astaire Dance Studios - Manasquan', ' 2410 NJ-35', ' Manasquan', ' NJ 08736']
 2410 NJ-35,  Manasquan, NJ, 08736
Cant Find the address
http://www.dancinginhighlandpark.com/IMGaller

Cant Find the address


In [5]:
#df1.to_excel("outputDance.xlsx")

# creating a dataframe for Dance
df1


,name,type,url,phoneNo,address,location
0,Dance Theatre of New Jersey,Dance Classes,http://www.dancetheatreofnewjersey.com/,3,"112 Bauer Dr, Oakland, NJ, 07436","[41.0061708789771, -74.240482416169]"
1,Newark School of the Arts,Dance Classes,http://www.newarkschoolofthearts.org/,,"89 Lincoln Park, Newark, NJ, 07102","[40.7256942631579, -74.1794780526316]"
2,Undercover Dance Factory,Dance Classes,http://www.undercoverdancefactory.com/home.html,,"2 Carlton Ave, East Rutherford, NJ, 07073","[40.8432, -74.113414]"
3,Union City Music & Dance Academy,Dance Classes,http://unioncitymusicdance.com/,,"118 40th St, Union City, NJ, 07087","[40.7742930408163, -74.0213140816327]"
4,Broadway Performing Arts,Dance Classes,http://broadwayperformingarts.net/,9737482787,"357 Broad St %231, Bloomfield, NJ, 07003",NaN
5,Artists In Motion Dance,Dance Classes,http://artistsinmotionnj.com/,4,"3562 NJ-27 N Suite 123, Kendall Park, NJ, 08824",NaN
6,Salsa Fever On 2 Dance Academy,Dance Classes,http://www.salsafeveron2.com/,,"83 Franklin St, Jersey City, NJ, 07307","[40.7420044285714, -74.0483392857143]"
7,Just Dancin',Dance Classes,http://justdancinnj.com/,,"187 Bloomfield Ave, Bloomfield, NJ, 07003","[40.7811202857143, -74.1905645714286]"
8,Professional Arts Academy,Dance Classes,http://www.professionalartsacademy.com/,,"9021 River Rd, North Bergen, NJ, 07047","[40.8052916432349, -73.9933710630243]"
9,Shiamak Davar Bollywood dance academy,Dance Classes,http://www.shiamak.com/newyork/register,2018999799,"488 Oak Tree Ave, South Plainfield, NJ, 07080",NaN


## Google Music

In [10]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&tbm=lcl&sxsrf=ACYBGNTdFaDd4jAeC6lZKeILHtxz3pMx4A%3A1574389394556&ei=kkbXXanHIeSV_Qbz-a_wAg&q=kids+music+classes+new+jersey&oq=kids+music+classes+new+jersey&gs_l=psy-ab.12...0.0.0.163915.0.0.0.0.0.0.0.0..0.0....0...1c..64.psy-ab..0.0.0....0.51R-_vCUfl4#rlfi=hd:;si:;mv:[[41.0328425,-73.9399237],[40.1005597,-74.3127616]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!1m4!1u2!2m2!2m1!1e1!1m4!1u16!2m2!16m1!1e1!1m4!1u16!2m2!16m1!1e2!2m1!1e2!2m1!1e3!2m1!1e16!3sIAE,lf:1,lf_ui:2'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

22
[{'name': 'So i Heard Music Gillette'}, {'name': 'In Tune Maplewood'}, {'name': 'The Institute of Music for Children'}, {'name': 'Little Rockers - Cool Kids Music Classes'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'Little Rockers - cool kids music classes & parties'}, {'name': 'Rockness Music of Madison'}, {'name': 'Musical Munchkins of New Jersey'}, {'name': 'Music Together of Bergen County'}, {'name': 'Music Together of Summit'}, {'name': 'New Jersey Youth Chorus'}, {'name': 'Edison School of Music and Dance'}, {'name': 'Rockness Music'}, {'name': 'Little Notes Music School'}, {'name': "Sharon's Studio of Dance & Music"}, {'name': 'School of Rock Chatham'}, {'name': 'Elefante Music & School for the Performing Arts'}, {'name': 'Joy Kids Music & Art (Joy Performing Art Center)'}, {'name': 'Rhythm Kids'}, {'name': 'Westerhoff School of Music & Art'}, {'name': 'Valencia School of Music'}, {'name': 'Rockness Music of Scotch Plains'}]


In [11]:
Data = []
geolocator = Nominatim(user_agent="my-application")
pageData = soup.find_all(class_='std')

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12,14,16,18] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Music Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            print(dataframe["url"])
        except:
            print('Cant Find the url')
            
        
        PhoneData = tr.find('span').text.split(' ')
        #print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
                    #print(orgAddress)
            try:
                locForLatLong = dataframe["address"] #+ dataframe["city"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
                
        top_result.append(dataframe)

#print(top_result)
df2=pd.DataFrame(top_result)



1
/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=5KjmXdKgOeWb5wKS9KzADw&start=20&sa=N&ved=0ahUKEwjSov20jZrmAhXlzVkKHRI6C_gQ8tMDCMYC
2
start=20
start=40
https://www.google.com/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=5KjmXdKgOeWb5wKS9KzADw&start=40&sa=N&ved=0ahUKEwjSov20jZrmAhXlzVkKHRI6C_gQ8tMDCMYC
20
http://www.musicplaystudios.com/
Music+Play+Studios,+33+River+Dr+S,+Jersey+City,+NJ+07310
['Music Play Studios', ' 33 River Dr S', ' Jersey City', ' NJ 07310']
 33 River Dr S,  Jersey City, NJ, 07310
33, River Drive South, Newport, Jersey City, Hudson County, New Jersey, 07310, United States of America
(40.7277594285714, -74.0321289795918)
http://musictogetherfamily.com/
Music+Together+Family+%26+Friends,+56+NJ-12,+Flemington,+NJ+08822
['Music Together Family %26 Friends', ' 56 NJ-12', ' Flemington', ' NJ 08822']
 56 NJ-12,  Flemington, NJ, 08822
Frenchtown - Flemington Road, Flemington, Hunterdon County, New Jersey, 

15, John Street, Bound Brook, Somerset County, New Jersey, 08805, United States
(40.5618427634078, -74.5334232548041)
https://kidsmusicround.com/class-location/bridgewater/
https://www.calderoneschoolofmusic.com/
Calderone+School+of+Music,+34+Ridgedale+Ave,+East+Hanover,+NJ+07936
['Calderone School of Music', ' 34 Ridgedale Ave', ' East Hanover', ' NJ 07936']
 34 Ridgedale Ave,  East Hanover, NJ, 07936
34, Ridgedale Avenue, East Hanover, Morris County, New Jersey, 07936, United States of America
(40.8139188578731, -74.3905728424745)
http://www.musicnotesacademy.com/
Music+Notes+Academy,+505+NJ-18,+East+Brunswick,+NJ+08816
['Music Notes Academy', ' 505 NJ-18', ' East Brunswick', ' NJ 08816']
 505 NJ-18,  East Brunswick, NJ, 08816
505, NJ 18, Tanners Corner, East Brunswick Township, Middlesex County, New Jersey, 08816, United States
(40.4407724010877, -74.3943018714253)
http://musicplaystudios.com/
Music+Together,+6040+Boulevard+E,+West+New+York,+NJ+07093
['Music Together', ' 6040 Boulev

Cant Find the address
https://montclairartmuseum.org/learn-yard-school.php
Yard+School+of+Art,+3+S+Mountain+Ave,+Montclair,+NJ+07042
['Yard School of Art', ' 3 S Mountain Ave', ' Montclair', ' NJ 07042']
 3 S Mountain Ave,  Montclair, NJ, 07042
Montclair Art Museum, 3, South Mountain Avenue, Montclair, Essex County, New Jersey, 07042, United States
(40.81860135, -74.2242380490856)
http://www.jcartschool.com/
Jersey+City+Art+School,+29+McWilliams+Pl,+Jersey+City,+NJ+07302
['Jersey City Art School', ' 29 McWilliams Pl', ' Jersey City', ' NJ 07302']
 29 McWilliams Pl,  Jersey City, NJ, 07302
29, McWilliams Place, Newport, Jersey City, Hudson County, New Jersey, 07302, United States of America
(40.7277296842105, -74.0440225526316)
http://readytopaint.com/
Ready+to+Paint,+424+Main+St,+Chatham,+NJ+07928
['Ready to Paint', ' 424 Main St', ' Chatham', ' NJ 07928']
 424 Main St,  Chatham, NJ, 07928
Main Street, Chatham, Morris County, New Jersey, 07928, United States of America
(40.744563, -74.

21, Pennington Road, Pennington, Mercer County, New Jersey, 08534, United States
(40.33088, -74.798377)
http://www.bassartsstudio.com/
Bass+Arts+Studio+LLC,+202+Park+St,+Montclair,+NJ+07042
['Bass Arts Studio LLC', ' 202 Park St', ' Montclair', ' NJ 07042']
 202 Park St,  Montclair, NJ, 07042
202, Park Street, Montclair, Essex County, New Jersey, 07042, United States of America
(40.8266568463764, -74.209211014832)
http://www.njschoolofmusic.com/
New+Jersey+School+of+Music+-+Cherry+Hill,+605+NJ-70,+Cherry+Hill,+NJ+08002
['New Jersey School of Music - Cherry Hill', ' 605 NJ-70', ' Cherry Hill', ' NJ 08002']
 605 NJ-70,  Cherry Hill, NJ, 08002
State Route 70, Cherry Hill Township, Camden County, New Jersey, 08002, United States
(39.919492, -75.034166)
10
start=100
start=120
https://www.google.com/search?q=kids+music+classes+new+jersey&rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=5KjmXdKgOeWb5wKS9KzADw&start=120&sa=N&ved=0ahUKEwjSov20jZrmAhXlzVkKHRI6C_gQ8tMDCMYC
20
https://waynenj.b2rmusic.com

485, Brick Boulevard, Brick Township, Ocean County, New Jersey, 08723, United States of America
(40.0472891809768, -74.1423454259753)
http://www.artist-people.com/
Artist+People+%7C+Sip+and+Paint+Studio+%7C+East+Brunswick,+712+NJ-18,+East+Brunswick,+NJ+08816
['Artist People %7C Sip and Paint Studio %7C East Brunswick', ' 712 NJ-18', ' East Brunswick', ' NJ 08816']
 712 NJ-18,  East Brunswick, NJ, 08816
712, NJ 18, Tanners Corner, East Brunswick Township, Middlesex County, New Jersey, 08816, United States of America
(40.4290995, -74.3839451666667)
http://tinygreenhousejc.com/
Tiny+Greenhouse,+498+Jersey+Ave,+Jersey+City,+NJ+07302
['Tiny Greenhouse', ' 498 Jersey Ave', ' Jersey City', ' NJ 07302']
 498 Jersey Ave,  Jersey City, NJ, 07302
Jersey Avenue, Newport, Jersey City, Hudson County, New Jersey, 07302, United States of America
(40.7212653, -74.0463542)
http://artacademynj.com/
Academy+of+Art,+727+Raritan+Ave,+Highland+Park,+NJ+08904
['Academy of Art', ' 727 Raritan Ave', ' Highland 

GeocoderTimedOut: Service timed out

In [ ]:
#df.to_excel("outputMusic.xlsx")

# Creating a dataframe for Music
df2

## Google Sports

In [ ]:
url = 'https://www.google.com/search?rlz=1C1CHBF_enUS853US853&sz=0&tbm=lcl&ei=05bdXaEyoqX9BoGbkuAK&q=kids+sports+classes+new+jersey&oq=kids+sports+classes+new+jersey&gs_l=psy-ab.12...13265.16654.0.22362.7.7.0.0.0.0.83.504.7.7.0....0...1c.1.64.psy-ab..0.0.0....0.EEUl6U13Yy0#rlfi=hd:;si:;mv:[[41.1122716,-73.93472659999999],[40.3136717,-74.6914486]];tbs:lrf:!1m4!1u3!2m2!3m1!1e1!2m1!1e3!3sIAE,lf:1,lf_ui:3'
browser.visit(url)

browser.find_by_name('btnG').first.click()

html = browser.html
soup = BeautifulSoup(html, 'html.parser')
time.sleep(5)

Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

print(len(Data))
top_result = []
    
for tr in Data:
    dataframe = {}
    dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
    #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
    top_result.append(dataframe)
print(top_result)

In [ ]:
Data = []
geolocator = Nominatim(user_agent="my-application")
pageData = soup.find_all(class_='std')
#print( soup.find("div", class_="std")["aria-label"] )

print(len(pageData))
for tr in pageData:
    
    pLink = str(tr.find('a')['href'])
    print(pLink)

top_result = []
list1 = [2, 4, 6,8,10,12] 
for x in (list1):
    prePage = x
    print(prePage)
    startPage = "start="+ str(prePage) +"0"
    print(startPage)
    replacePage = "start="+ str(prePage+2) +"0"
    print(replacePage)
    pLink = pLink.replace(startPage,replacePage)
    pageLink = "https://www.google.com" + pLink
    print(pageLink)
    browser.visit(pageLink)

  
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    Data = soup.find_all("div",attrs = {"class": "VkpGBb"})

    print(len(Data))
    
    for tr in Data:
        
        dataframe = {}
        dataframe["name"] = (tr.find("div",attrs = {"class": "dbg0pd"})).text.replace('\n', ' ')
        #dataframe["Reviews"] = (tr.find('span', {'class':'BTtC6e'}))
        dataframe["type"] = "Sports Classes"
        try:
            dataframe["url"] = (tr.find("a",attrs = {"class": "yYlJEf L48Cpd"})['href'])
            
            print(dataframe["url"])
        except:
            print('Cant Find the url')
            
        
        PhoneData = tr.find('span').text.split(' ')
        print(PhoneData)
        lPhoneData = len(PhoneData)
        phoneNo = PhoneData[lPhoneData-2]+ PhoneData[lPhoneData-1]
        phoneNo = re.sub("\D", "", phoneNo)
        dataframe["phoneNo"] = phoneNo
        
        dirClass = (tr.find('div', {'class':'UbRuwe'}))
        #print(dirClass)
        if dirClass != None :
        #if hasattr('a', 'cursor:pointer'):
            rawAddress = (tr.find("a",attrs = {"class": "yYlJEf VByer"})["data-url"])
            
            orgAddress = rawAddress.split('/')[4].strip()
            print(orgAddress)
            orgAddress = orgAddress.replace('+',' ')
            orgAddress = orgAddress.split(',')
            
            print(orgAddress)
            lenAddress = len(orgAddress)
            
            if lenAddress == 4:
                ZipCode = orgAddress[3].split(' ')
                #print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[1]+', ' + orgAddress[2]+', ' + ZipCode[1]+', ' + ZipCode[2]
                #dataframe["city"] = orgAddress[2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
                    
            if lenAddress > 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(ZipCode)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-3]+', '+orgAddress[lenAddress-2]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-2]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                
            if lenAddress < 4 :
                ZipCode = orgAddress[lenAddress-1].split(' ')
                print(orgAddress)
                #print (string_in_slashes.split(','))
            
                dataframe["address"] = orgAddress[lenAddress-2]+', '+orgAddress[lenAddress-1]+', '+ ZipCode[1]+', '+ZipCode[2]
                #dataframe["city"] = orgAddress[lenAddress-1]
                #dataframe["state"] = ZipCode[1]
                #if len(ZipCode) > 2:
                    #dataframe["zipcode"] = ZipCode[2]
                    #dataframe["Zipcode"] = orgAddress[4]
            
                    #print(orgAddress)
                    
            try:
                locForLatLong = dataframe["address"]# + dataframe["city"]
                print(locForLatLong)
                location = geolocator.geocode(locForLatLong, timeout=10)
                print(location.address)
                print((location.latitude,location.longitude))
                dataframe["location"] = [location.latitude,location.longitude]
                
            except AttributeError:
                print('Cant Find the address')
                
        top_result.append(dataframe)

#print(top_result)
df3=pd.DataFrame(top_result)



In [ ]:
#df.to_excel("outputSports.xlsx")

# creating a dataframe for sports
df3


## Merged all three dataframe into one DF

In [ ]:
# merging all 3 dataframe for music, Dannce and Sports

df = df1.append(df2, ignore_index=True)
df = df.append(df3, ignore_index=True)

# Cleaning the dataframet to drop null values for location

df = df.dropna(axis=0, subset=['location'])
df

In [ ]:
# exporting to excel

df.to_excel("outputKidsActivities.xlsx") 

In [ ]:
# reseting the index

df=df.reset_index()
df.head()

In [ ]:
# renaming the column to _id
df = df.drop(df.index[0])
df = df.rename(columns={'index':'_id'})
#df = df.drop(columns=['Reviews'])
#df = df.drop(columns=['_id'])
df.head()

In [ ]:
# adding to directory
split_data = df.to_dict('records')
split_data

In [ ]:
# Ensure that all types are objects
df.dtypes

In [ ]:
# Set up the MongoDB connection through pymongo

myclient = pymongo.MongoClient("mongodb://localhost:27017/")

# Create DB

db = myclient["KidsRidedb3"]

# Create collection and insert all the data into the MongoDB

mycol = db["activityData9"]
mycol.insert_many(split_data)